Installing the BeautifulSoup4 and lxml packages

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} beautifulsoup4
!conda install --yes --prefix {sys.prefix} lxml

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.5.15  |                1         134 KB

The following packages will be UPDATED:

    ca-certificates: 2019.5.15-0 --> 2019.5.15-1


ca-certificates-2019 | 134 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.4                 |   py36hefd8a0e_0         1.5 MB

The following packages will be UPDATED:



Importing the necessary packages/libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Getting the table from the Wikipedia page

In [3]:
source= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(source, 'lxml')
data_table=soup.find('table', class_="wikitable sortable")

Converting the html table into a Pandas dataframe

In [4]:
table_rows = data_table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    #row = [tr.text for tr in td]
    #l.append(row)
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)
raw_table=pd.DataFrame(l, columns=["PostCode", "Borough", "Neighbourhood"])
raw_table.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping the rows with borough not assigned

In [5]:
#dropping the rows with borough not assigned
raw_table.drop(raw_table[raw_table.Borough=="Not assigned"].index, inplace=True)
raw_table.reset_index(drop=True, inplace=True)
raw_table.shape

(211, 3)

For 'Not assigned' neighbourhoods, making their Borough as their Neighbourhood

In [13]:
#for Not Assigned neighbourhoods, making their Borough as their Neighbourhood
for index, row in raw_table.iterrows():
    if row['Neighbourhood']=="Not assigned":
        row['Neighbourhood']=row['Borough']
raw_table.head(20)

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Merging rows with the same Post Codes

In [11]:
#raw_table,merging rows
final_table=raw_table.groupby(['PostCode','Borough'],as_index=False, sort=False).agg(','.join)
final_table['Neighbourhood']=set(final_table['Neighbourhood'])
final_table.head()


,PostCode,Borough,Neighbourhood
0,M3A,North York,Islington Avenue
1,M4A,North York,"The Danforth West,Riverdale"
2,M5A,Downtown Toronto,"Commerce Court,Victoria Hotel"
3,M6A,North York,Berczy Park
4,M7A,Queen's Park,Willowdale West


Displaying the final  shape of the dataframe

In [12]:
final_table.shape

(103, 3)